In [28]:
import math
import pandas as pd
from gurobipy import *
import time
beam_cost = 1380
welding_cost = 156


In [61]:
def generate_model(init_rods, rods, num_beams, beam_length, beams, time_limit, input_file=None):
    m = Model("rods")
    y = []
    r = [[0 for j in range(len(rods))] for i in range(num_beams)]
    x = [[0 for j in range(len(rods))] for i in range(num_beams)]
    M=99999
    for i in range(num_beams):
        y.append(m.addVar(vtype=GRB.BINARY, name="y{}".format(i))) # 1 if using beam i else 0

        for j in range(len(rods)):
            x[i][j] = (m.addVar(vtype=GRB.BINARY, name="x{},{}".format(i,j)))

    for i in range(num_beams):
        for j in range(len(rods)):
            r[i][j] = (m.addVar(vtype=GRB.INTEGER, lb=0, name = "r{}, {}".format(i,j))) #Length of rod j assigned to beam i


    m.setObjective(1380*quicksum(y)+156*(quicksum([x[i][j] for i in range(num_beams) for j in range(len(rods))])-len(rods)), GRB.MINIMIZE)


    for i in range(num_beams):
        m.addConstr(beam_length*y[i] 
                        - (quicksum([r[i][j] for j in range(len(rods))])
                           + quicksum([5*x[i][j] for j in range(len(rods))])) >= 0)


    for j in range(len(rods)):        
        m.addConstr(quicksum([r[i][j] for i in range(num_beams)]) == rods[j])
    m.addConstr(quicksum([r[i][j] for i in range(num_beams)]) == rods[j])

    #m.addConstr(quicksum(y) >= 233)   
    m.addConstr(quicksum([x[i][j] for i in range(num_beams) for j in range(len(rods))]) <= 542)

    for i in range(num_beams):
        for j in range(len(rods)):
            m.addConstr(beam_length*x[i][j] >= r[i][j])

    for i in range(num_beams):
        for j in range(len(rods)):
            m.addConstr((1/beam_length)*x[i][j] <= r[i][j])

    for j in range(len(rods)):
        m.addConstr(quicksum([x[i][j] for i in range(num_beams)]) <= 6)
        
    if input_file:
        df = pd.read_csv(input_file)
        array =df.values
        array = [[math.ceil(float(j)) for j in i]for i in array]
        #Starting initial r values at lengths from part A
        for i in range(num_beams):
            for j in range(len(rods)):
                if i<230 and array[j][i]>0 :
                    m.addConstr(r[i][j] == array[j][i])
                    m.addConstr(x[i][j] == 1)
                    
                r[i][j].start = array[j][i]

        m.update()

        #starting initial x values at binaries related to r
        for i in range(num_beams):
            for j in range(len(rods)):
                if r[i][j].start > 0:
                    x[i][j].start = 1
                else:
                    x[i][j].start = 0

        m.update()

        for i in range(num_beams):
            if sum([x[i][j].start for j in range(len(rods))]) > 0:
                y[i].start = 1
            else:
                y[i].start = 0

    m.update()
    1380*sum(y[i].start for i in range(num_beams))+156*(sum([x[i][j].start for i in range(num_beams) for j in range(len(rods))])-len(rods))
       
    
    
    m.setParam('TimeLimit', time_limit)
    m.setParam('MIPFocus',1)
    m.optimize()
    return m, r

In [ ]:
number_sets = 1 # Number of equal sets of rods

num_beams = 251 # int(360/number_sets)
beam_length = 12005

beams = [beam_length]*num_beams # Initiate beams
models = []
r_vars = []
total_obj = 0

# RUN ALL SETS BELOW

for current_set in range(number_sets):
    init_rods = pd.read_csv('rod_list_2.csv')
    init_rods = list(init_rods[init_rods.index % number_sets == current_set]['Rod length']) 
    rods = [math.ceil(rod) for rod in init_rods]

    model, r = (generate_model(init_rods, rods, num_beams, beam_length,
                               beams, time_limit=200,input_file='rod_assignments_iteration10_output.csv'))
    models.append(model)
    r_vars.append(r)
    total_obj = total_obj + model.objVal
print(total_obj)   
    
"""
current_set = 0
init_rods = pd.read_csv('rod_list.csv')
init_rods = list(init_rods[init_rods.index % number_sets == current_set]['Rod length']) 
rods = [math.ceil(rod) for rod in init_rods]

model = generate_model(init_rods, rods, num_beams, beam_length, beams, time_limit=10)
# Multiply obj of set 0 by 6 to have a worst-case scenario bound. Run all 6 to have a better solution.
"""

Changed value of parameter TimeLimit to 200.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter MIPFocus to 1
   Prev: 0  Min: 0  Max: 3  Default: 0
Optimize a model with 201587 rows, 200047 columns and 900326 nonzeros
Variable types: 0 continuous, 200047 integer (100149 binary)
Coefficient statistics:
  Matrix range     [8e-05, 1e+04]
  Objective range  [2e+02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]

Loaded MIP start with objective 346380

Presolve removed 187782 rows and 186526 columns
Presolve time: 0.69s
Presolved: 13805 rows, 13521 columns, 60771 nonzeros
Variable types: 0 continuous, 13521 integer (6771 binary)

Root relaxation: objective 3.339244e+05, 8166 iterations, 0.22 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 333924.387    0   41 346380.000 333924.387  3.60%     -    1s
H    0 

In [ ]:
335484.000

In [21]:
models

[<gurobi.Model MIP instance rods: 8236 constrs, 8100 vars, Parameter changes: TimeLimit=3600.0, MIPFocus=1, LogFile=gurobi.log, CSIdleTimeout=1800>,
 <gurobi.Model MIP instance rods: 8236 constrs, 8100 vars, Parameter changes: TimeLimit=3600.0, MIPFocus=1, LogFile=gurobi.log, CSIdleTimeout=1800>,
 <gurobi.Model MIP instance rods: 8114 constrs, 7980 vars, Parameter changes: TimeLimit=3600.0, MIPFocus=1, LogFile=gurobi.log, CSIdleTimeout=1800>,
 <gurobi.Model MIP instance rods: 8114 constrs, 7980 vars, Parameter changes: TimeLimit=3600.0, MIPFocus=1, LogFile=gurobi.log, CSIdleTimeout=1800>,
 <gurobi.Model MIP instance rods: 8114 constrs, 7980 vars, Parameter changes: TimeLimit=3600.0, MIPFocus=1, LogFile=gurobi.log, CSIdleTimeout=1800>,
 <gurobi.Model MIP instance rods: 8114 constrs, 7980 vars, Parameter changes: TimeLimit=3600.0, MIPFocus=1, LogFile=gurobi.log, CSIdleTimeout=1800>]

Changed value of parameter TimeLimit to 300.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter MIPFocus to 1
   Prev: 0  Min: 0  Max: 3  Default: 0
Optimize a model with 16094 rows, 15960 columns and 71520 nonzeros
Variable types: 0 continuous, 15960 integer (8040 binary)
Coefficient statistics:
  Matrix range     [8e-05, 1e+04]
  Objective range  [2e+02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e+00, 1e+04]
Found heuristic solution: objective 85560.000000
Presolve removed 1 rows and 0 columns
Presolve time: 0.10s
Presolved: 16093 rows, 15960 columns, 71400 nonzeros
Variable types: 0 continuous, 15960 integer (8040 binary)

Root relaxation: objective 5.270370e+04, 13078 iterations, 0.83 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 52703.7001    0  105 85560.0000 52703.7001  38.4%     -    1s
H    0     0  

In [161]:
m.setParam('TimeLimit', 300)
m.setParam('MIPFocus',1)
m.optimize()

Changed value of parameter TimeLimit to 300.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter MIPFocus to 1
   Prev: 0  Min: 0  Max: 3  Default: 0
Optimize a model with 16336 rows, 16200 columns and 72600 nonzeros
Variable types: 0 continuous, 16200 integer (8160 binary)
Coefficient statistics:
  Matrix range     [8e-05, 1e+04]
  Objective range  [2e+02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e+00, 1e+04]
Found heuristic solution: objective 82800.000000
Presolve removed 1 rows and 0 columns
Presolve time: 0.13s
Presolved: 16335 rows, 16200 columns, 72480 nonzeros
Variable types: 0 continuous, 16200 integer (8160 binary)

Root relaxation: objective 5.405910e+04, 13209 iterations, 0.71 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 54059.1004    0   81 82800.0000 54059.1004  34.7%     -    1s
H    0     0  

In [ ]:
0
56760.0000
1
2
3
4
5

In [ ]:
114300.000 - best bound is 107388.830
113676.000
113232.000 -- best bound is  107071.447
112920.000
112920 -- best bound is 106093.664

In [55]:
for model in models:
    output = [[0 for i in range(num_beams)] for j in range(len(rods))]
    for j in range(len(rods)):
        for i in range(num_beams):
            output[j][i] = r[i][j].x

In [56]:
pd.DataFrame(output).to_csv('rod_assignments_iteration11_output.csv')
# Solution is 340848